In [15]:
COUNT = [10]  #global variable


class Node:
    def __init__(self, newdata):
        self.data = newdata  
        self.is_leaf = self.check_data()  
        self.right = None
        self.left = None

    def check_data(self):    #check node or leaf
        '''Checks whether given char is a node or a leaf'''
        node_sign = ["*", "|", "+", "?", "(", ")", "."]
        for i in node_sign:
            if self.data == i:
                return False
        return True

    def __str__(self):      #print
        '''To print the data in convenient way'''
        return self.data


def RegExpToString(exp):  
    op = ["*", "|", "+", "?", ")"]
    output = []
    counter = 0
    for i in exp:
        marker = False
        for o in op:
            if o == i:
                if output:
                    if output[-1] == ".":
                        output.pop()
        if i == "|":
            marker = True
        elif i == "(":
            marker = True
        output.append(i)
        if not marker:
            output.append(".")
    if output[-1] != ".":
        output.append(".")
    output.append("#") 
    return output


def check_precedence(char, top):
    precedence = {'*': 2, '?': 2, '+': 2, '.': 1, '|': 0, '(': 3}
    if precedence[char] < precedence[top]:
        return True
    return False


def RegExp2Postfix(exp):  #turn to postfix notation
    exp = RegExpToString(exp)
    output2 = []
    op = []
    for i in exp:
        buffer = Node(i)
        if buffer.is_leaf:
            output2.append(buffer)
        else:
            if buffer.data == "(":
                op.append(buffer)
            elif buffer.data == ")":
                while op[-1].data != "(":
                    output2.append(op.pop())
                if op[-1].data == "(":
                    op.pop()
            else:
                while (op and check_precedence(buffer.data, op[-1].data)
                       and op[-1].data != '('):
                    output2.append(op.pop())
                op.append(buffer)
    while op:
        output2.append(op.pop())
    return output2


class TreeNode:
    def __init__(self, right, left, op, id):
        self.right = right
        self.left = left
        self.data = op
        self.nullable = False
        self.id = id
        self.prev_position = []
        self.last_prev_position = []
        self.follow_list = []

    def PrintTree(self):  #printing the tree
        if self.left:
            self.left.PrintTree()
        print("Node:", self.data,"Nullable:", self.nullable,"ID:", self.id,"Firstpos:", self.prev_position,"Lastpos:",
              self.last_prev_position,"Followpos:", self.follow_list)
        if self.right:
            self.right.PrintTree() 

    def Nullable(self):     #check if contains any empty string/epsilon
        if self.data.data == "|":
            if self.left:
                if self.left.Nullable():
                    self.nullable = True
            if self.right:
                if self.right.Nullable():
                    self.nullable = True
        elif self.data.data == ".":
            if self.left and self.right:
                if self.left.Nullable() and self.right.Nullable():
                    self.nullable = True
        elif self.data.data == "?" or self.data.data == "*":
            self.nullable = True
        elif self.data.data == "":
            self.nullable = True

    def is_nullable(self):    
        if self.left:
            self.left.Nullable()
            self.left.is_nullable()
        if self.right:
            self.right.Nullable()
            self.right.is_nullable()

    def first(self):            #firstpos
        if self.prev_position:        #blocks from repeating the code for the same node several times
            return self.prev_position
        if self.data.data == "|":  
            if self.left:      
                for i in self.left.first():
                    if not i in self.prev_position:
                        self.prev_position.append(i)
            if self.right:
                for i in self.right.first():
                    if not i in self.prev_position:
                        self.prev_position.append(i)
            return self.prev_position
        elif self.data.data == ".":
            if self.left:
                if self.left.nullable:  
                    for i in self.left.first():
                        if not i in self.prev_position:
                            self.prev_position.append(i)
                    if self.right:
                        for i in self.right.first():
                            if not i in self.prev_position:
                                self.prev_position.append(i)
                    return self.prev_position
                else:
                    return self.left.first()
        elif self.data.data == "?" or self.data.data == "*" or self.data.data == "+":
            if self.right:  
                return self.right.first()
        elif self.data.data == "":
            return self.prev_position
        else:
            self.prev_position.append(self.id)
            return self.prev_position

    def assign_first(self):
        if self.left:
            self.left.assign_first()
        t = self.first()  
        self.prev_position = t
        if self.right:
            self.right.assign_first() 

    def last(self):
        if self.last_prev_position: 
            return self.last_prev_position
        if self.data.data == "|": 
            if self.left:  
                for i in self.left.last():
                    if not i in self.last_prev_position:
                        self.last_prev_position.append(i)
            if self.right:
                for i in self.right.last():
                    if not i in self.last_prev_position:
                        self.last_prev_position.append(i)
            return self.last_prev_position
        elif self.data.data == ".":
            if self.right:
                if self.right.nullable:  
                    for i in self.right.last():
                        if not i in self.last_prev_position:
                            self.last_prev_position.append(i)
                    if self.left:
                        for i in self.left.last():
                            if not i in self.last_prev_position:
                                self.last_prev_position.append(i)
                    return self.last_prev_position 
                else:
                    return self.right.last() 
        elif self.data.data == "?" or self.data.data == "*" or self.data.data == "+":
            if self.right:
                return self.right.last() 
        elif self.data.data == "":
            return self.last_prev_position
        else:
            self.last_prev_position.append(self.id)
            return self.last_prev_position

    def assign_last(self):
        if self.left:
            self.left.assign_last()
        t = self.last()
        self.last_prev_position = t
        if self.right:
            self.right.assign_last()

    def find_leaf(self, id):
        ret_val = None
        if self.left: 
            ret_val = self.left.find_leaf(id)
            if ret_val:
                return ret_val
        if self.data.is_leaf and self.id == id:
            return self 
        if self.right:
            ret_val = self.right.find_leaf(id)
            if ret_val:
                return ret_val
        return ret_val

    def update_follow(self, id, second_param):        #add next input as follow
        if self.left:  
            self.left.update_follow(id, second_param)

        if self.data.is_leaf and self.id == id:
            for i in second_param:  
                if not i in self.follow_list:            #not to store duplicates
                    self.follow_list.append(i)
        if self.right:
            self.right.update_follow(id, second_param)

    def follow(self):            #follow conditions
        if self.data.data == ".":
            for i in self.left.last_prev_position:
                self.update_follow(i, self.right.prev_position)
        elif self.data.data == "*" or self.data.data == "+":
            for i in self.last_prev_position:
                self.update_follow(i, self.prev_position)

    def follow_for_each_node(self):   #add follow to each node
        if self.left:  
            self.left.follow_for_each_node()
        self.follow()
        if self.right:
            self.right.follow_for_each_node()




class SyntaxTree:   
    def __init__(self):
        self.root = None

    def create(self, sentence):
        postfix = RegExp2Postfix(sentence)
        stack = []
        counter = 1
        for i in postfix:
            if not i.is_leaf:
                if stack:  
                    right = stack.pop()
                else:
                    right = TreeNode(None, None, Node(""), None)
                if stack:
                    left = stack.pop()
                else:
                    left = TreeNode(None, None, Node(""), None)
                stack.append(TreeNode(right, left, i, None))
            else:
                stack.append(TreeNode(None, None, i, counter))
                counter = counter + 1

        self.root = stack.pop()
        self.root.is_nullable()
        self.root.assign_first()
        self.root.assign_last()
        self.root.follow_for_each_node()

    def PrintTree(self):
        self.root.PrintTree()


class DFA:
    def __init__(self):
        self.Dstate = {}
        self.Dtran = {}
        self.final_states = []

    def construct(self, tree, string):
        s0 = tree.root.prev_position
        self.Dstate[tuple(s0)] = 'unmarked'
        i = 0
        keys = list(self.Dstate.keys())     #list of tuples of keys
        prev_list = []  
        while i < len(keys):
            if self.Dstate[keys[i]] == 'unmarked':
                self.Dstate[keys[i]] = chr(i + 65)
                for a in string:
                    j = 0
                    while j < len(keys) and keys[j] != 'unmarked':
                        prev_list.clear()
                        for l in keys[j]:
                            leaf = tree.root.find_leaf(l)
                            if not leaf:
                                print("There is no such letter in the alphabeth")
                                break
                            if leaf.data.data == a.data:
                                if leaf.follow_list:
                                    prev_list = prev_list + leaf.follow_list  
                                    if not tuple(prev_list) in self.Dstate:
                                        self.Dstate[tuple(prev_list)] = 'unmarked'
                                self.Dtran[(self.Dstate[keys[i]],a.data)] = tuple(prev_list)
                        j = j + 1
                keys = list(self.Dstate.keys())
            i = i + 1

        for i in list(self.Dtran.keys()):     #state->transition
            if self.Dtran[i] in self.Dstate.keys():
                self.Dtran[i] = self.Dstate[self.Dtran[i]]

        for i in self.Dtran.keys():   #finalstate/s
            if not self.Dtran[i]:
                self.final_states.append(i[0])

    def printDFA(self):
        print("State Table")
        print("")
        for i in self.Dstate.keys():
            print("State:", self.Dstate[i],"->", i)
        print("")
        print("Transition Table")
        print("")
        for i in self.Dtran.keys():
            print(i, self.Dtran[i])



def main(regex):
    tree = SyntaxTree()
    tree.create(regex)
    print("Syntax tree")
    print("")
    tree.PrintTree()
    print("")
    print("DFA")
    print("")
    dfa = DFA()
    postfix = RegExp2Postfix(regex)
    dfa.construct(tree, postfix)
    dfa.printDFA()
    return dfa


main("(a*|b)*cd")

Syntax tree

Node:  Nullable: True ID: None Firstpos: [] Lastpos: [] Followpos: []
Node: * Nullable: True ID: None Firstpos: [1, 2] Lastpos: [1, 2] Followpos: []
Node:  Nullable: True ID: None Firstpos: [] Lastpos: [] Followpos: []
Node: * Nullable: True ID: None Firstpos: [1] Lastpos: [1] Followpos: []
Node: a Nullable: False ID: 1 Firstpos: [1] Lastpos: [1] Followpos: [1, 2, 3]
Node: | Nullable: False ID: None Firstpos: [1, 2] Lastpos: [1, 2] Followpos: []
Node: b Nullable: False ID: 2 Firstpos: [2] Lastpos: [2] Followpos: [1, 2, 3]
Node: . Nullable: False ID: None Firstpos: [1, 2, 3] Lastpos: [5] Followpos: []
Node: c Nullable: False ID: 3 Firstpos: [3] Lastpos: [3] Followpos: [4]
Node: . Nullable: False ID: None Firstpos: [3] Lastpos: [5] Followpos: []
Node: d Nullable: False ID: 4 Firstpos: [4] Lastpos: [4] Followpos: [5]
Node: . Nullable: False ID: None Firstpos: [4] Lastpos: [5] Followpos: []
Node: # Nullable: False ID: 5 Firstpos: [5] Lastpos: [5] Followpos: []

DFA

State Tabl